In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

%matplotlib inline

# PREPARING DATA

In [ ]:
source_dir = "mydata/"
print(os.listdir(source_dir))

In [ ]:
list_dir = os.listdir(source_dir)
print(list_dir)

# iterasi untuk menemukan folder gambar
index=0
for i in range (len(source_dir)):
    if source_dir[i]=="images":
        index=i
        break

# isi dari image directory
index_image = os.listdir(source_dir+list_dir[index])
#print(type(imageIndex))
head = 10

# list kumpulan sampel
image_sample = []

In [ ]:
list_dir = os.listdir(source_dir) 
# iterasi untuk menemukan folder gambar
index=0
for i in range (len(list_dir)):
    if list_dir[i]=="images":
        index=i
        break

# isi dari image directory
index_image = os.listdir(list_dir[index])
head = 10

# list kumpulan sampel
image_sample = []


# CLEANING DATA

In [ ]:
doc = pd.read_csv(source_dir + "styles.csv", 
                 nrows=5000, 
                 error_bad_lines=False)
doc['image'] = doc.apply(lambda x: str(x['id']) + ".jpg", axis=1)
doc.drop(['season', 'year', 'usage', 'productDisplayName'], 
         axis=1, 
         inplace=True)
print(doc.shape)
doc.head(5)

In [ ]:
plt. figure(figsize=(7,4))
for i in range(head):
    plt.subplot(2,5,i+1)
    plt.imshow(cv2.imread(source_dir + "images/" + doc.iloc[i]['image']))
    plt.title(doc.iloc[i]['articleType'])
    plt.axis('off')
plt.show()

# PRE-PROCESSING DATA

In [ ]:
data_augmented = pd.DataFrame({
    'filename': doc['image'],
    'type'    : doc['articleType']
})

# total number of entries in the dataframe
total_baris = len(data_augmented)
print('jumlah baris total: ', total_baris)

In [ ]:
cloth_type = data_augmented['type'].unique().tolist()
total_type = len(cloth_type)
print(total_type)
print(cloth_type)
print(cloth_type[0])
print(cloth_type.index(cloth_type[0]))
data_augmented['number_types'] = data_augmented['type'].apply(lambda x: cloth_type.index(x) if x in cloth_type else 0)
data_augmented.head(10)

In [ ]:
data_augmented['type'].value_counts()

# BUILD MODEL 

In [ ]:
def image_data(start,stop):
    data_x = []
    data_y = []

    
    for i in range(start, stop): 
        image_source=source_dir+list_dir[index]+"/"+data_augmented.loc[i,'filename']
        image = cv2.imread(image_source,cv2.IMREAD_GRAYSCALE)
        try:
            image_resized = cv2.resize(image, dsize=(28, 28))
        except:
            print("Rejected Image: "+data_augmented.loc[i,'filename'])
            continue
        data_x.append(image_resized) 
        values_y = data_augmented.loc[i,'number_types']
        data_y.append(values_y)
                
    return data_x, data_y

In [ ]:
X=[]
Y=[]

start = 0
stop = int(0.001*total_baris) 

print(type(X))
print(X[0])
plt.title(cloth_type[Y[0]])
plt.imshow(X[0])

In [ ]:
X = np.array(X).reshape(-1, 28,28,1)
Y = np.array(Y)

X = X/255.0
Y = Y.reshape(len(X),)

In [ ]:
X=[]
Y=[]
start = 0
stop = total_baris
X,Y=image_data(start,stop)

X = np.array(X).reshape(-1, 28,28,1)
Y = np.array(Y)

X = X/255

Y = Y.reshape(len(X),)

print('Image Data Shape: ',X.shape)
print('Label Data Shape: ',Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 500)
print('Train Image Data Shape: ',X_train.shape)
print('Train Label Data Shape: ',Y_train.shape)
print('Test Image Data Shape: ',X_test.shape)
print('Test Label Data Shape: ',Y_test.shape)

In [ ]:
X_train, X_validate, Y_train, Y_validate = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 500)
print('Train Image Data Shape: ',X_train.shape)
print('Train Label Data Shape: ',Y_train.shape)
print('Validation Image Data Shape: ',X_validate.shape)
print('Validation Label Data Shape: ',Y_validate.shape)

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), 
                     activation='relu', 
                     input_shape=(28,28,1)))
    model.add(Conv2D(64, (3, 3), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, 
                    activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(cloth_type), 
                    activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='Adam', 
                  metrics=['accuracy'])
    return model

model=create_model()
print(model.summary())

# Train Model

In [ ]:
history = model.fit(X_train, 
                    Y_train, 
                    batch_size=128, 
                    epochs=50, 
                    verbose=1, 
                    validation_data=(X_validate, Y_validate))

In [ ]:
plt.plot(model.history.history['acc'],color='g')
plt.plot(model.history.history['loss'],color='r')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'loss'], loc='upper right')
plt.show()

# Saving Model

In [ ]:
model.save('f_recommend.h5')

In [ ]:
from tensorflow import keras
tflite_model = tf.keras.models.load_model('f_recommend.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
open("mdel.tflite", "wb").write(tflite_save)